In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
# Import Python libraries
import logging

# Import Pyomo units
from pyomo.environ import units as pyunits
from pyomo.environ import SolverFactory

from idaes.core import FlowsheetBlock

# Import IDAES cores
from idaes.core import LiquidPhase, VaporPhase, Component

from idaes.generic_models.properties.core.state_definitions import FTPx
from idaes.generic_models.properties.core.eos.ideal import Ideal
from idaes.generic_models.properties.core.phase_equil import SmoothVLE
from idaes.generic_models.properties.core.phase_equil.bubble_dew import \
        IdealBubbleDew,LogBubbleDew
from idaes.generic_models.properties.core.eos.ceos import Cubic, CubicType
from idaes.generic_models.properties.core.phase_equil.forms import fugacity,log_fugacity

from idaes.generic_models.properties.core.pure import RPP4
from idaes.generic_models.properties.core.pure import Perrys

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

In [3]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [4]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [5]:
data = pd.read_csv('BT_NRTL_dataset.csv')
print(data)

    temperature  liq_benzene  vap_benzene
0    365.500000     0.480953     0.692110
1    365.617647     0.462444     0.667699
2    365.735294     0.477984     0.692441
3    365.852941     0.440547     0.640336
4    365.970588     0.427421     0.623328
5    366.088235     0.442725     0.647796
6    366.205882     0.434374     0.637691
7    366.323529     0.444642     0.654933
8    366.441176     0.427132     0.631229
9    366.558824     0.446301     0.661743
10   366.676471     0.438004     0.651591
11   366.794118     0.425320     0.634814
12   366.911765     0.439435     0.658047
13   367.029412     0.435655     0.654539
14   367.147059     0.401350     0.604987
15   367.264706     0.397862     0.601703
16   367.382353     0.415821     0.630930
17   367.500000     0.420667     0.640380
18   367.617647     0.391683     0.598214
19   367.735294     0.404903     0.620432
20   367.852941     0.409563     0.629626
21   367.970588     0.389488     0.600722
22   368.000000     0.396789     0

### 2.3 Creating a generic property parameter block

The next step is to give all the parameter information required for the sub-libraries. Here we will specify the base units of our model, the bounds on the state properties and the reference conditions of our properties.

In [6]:
# Add properties parameter blocks to the flowsheet with specifications -- pull from .py file
# from idaes.generic_models.properties.core.examples.BT_VDW import configuration

In [7]:
#Make configuration from original Dictionary_Txy_diagrams_AG
configuration = {
    # Specifying components
    "components": {
        'benzene': {"type": Component,
                    "elemental_composition": {"C": 6, "H": 6},
                    "enth_mol_liq_comp": Perrys,
                    "enth_mol_ig_comp": RPP4,
                    "pressure_sat_comp": RPP4,
                    "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
                    "parameter_data": {
                        "mw": (78.1136E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (48.9e5, pyunits.Pa),  # [1]
                        "temperature_crit": (562.2, pyunits.K),  # [1]
                        "omega": 0.212,  # [1]
                        "dens_mol_liq_comp_coeff": {
                            '1': (1.0162, pyunits.kmol*pyunits.m**-3),  # [2] pg. 2-98
                            '2': (0.2655, None),
                            '3': (562.16, pyunits.K),
                            '4': (0.28212, None)},
                        "cp_mol_ig_comp_coeff": {
                            'A': (-3.392E1, pyunits.J/pyunits.mol/pyunits.K),  # [1]
                            'B': (4.739E-1, pyunits.J/pyunits.mol/pyunits.K**2),
                            'C': (-3.017E-4, pyunits.J/pyunits.mol/pyunits.K**3),
                            'D': (7.130E-8, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "cp_mol_liq_comp_coeff": {
                            '1': (1.29E2, pyunits.J/pyunits.kmol/pyunits.K),  # [2]
                            '2': (-1.7E-1, pyunits.J/pyunits.kmol/pyunits.K**2),
                            '3': (6.48E-4, pyunits.J/pyunits.kmol/pyunits.K**3),
                            '4': (0, pyunits.J/pyunits.kmol/pyunits.K**4),
                            '5': (0, pyunits.J/pyunits.kmol/pyunits.K**5)},
                        "enth_mol_form_liq_comp_ref": (
                            49.0e3, pyunits.J/pyunits.mol),  # [3]
                        "enth_mol_form_vap_comp_ref": (
                            82.9e3, pyunits.J/pyunits.mol),  # [3]
                        "pressure_sat_comp_coeff": {'A': (-6.98273, None),  # [1]
                                                    'B': (1.33213, None),
                                                    'C': (-2.62863, None),
                                                    'D': (-3.33399, None)}}},
        'toluene': {"type": Component,
                    "elemental_composition": {"C": 7, "H": 8},
                    "enth_mol_liq_comp": Perrys,
                    "enth_mol_ig_comp": RPP4,
                    "pressure_sat_comp": RPP4,
                    "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
                    "parameter_data": {
                        "mw": (92.1405E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (41e5, pyunits.Pa),  # [1]
                        "temperature_crit": (591.8, pyunits.K),  # [1]
                        "omega": 0.263,  # [1]
                        "dens_mol_liq_comp_coeff": {
                            '1': (0.8488, pyunits.kmol*pyunits.m**-3),  # [2] pg. 2-98
                            '2': (0.26655, None),
                            '3': (591.8, pyunits.K),
                            '4': (0.2878, None)},
                        "cp_mol_ig_comp_coeff": {
                            'A': (-2.435E1, pyunits.J/pyunits.mol/pyunits.K),  # [1]
                            'B': (5.125E-1, pyunits.J/pyunits.mol/pyunits.K**2),
                            'C': (-2.765E-4, pyunits.J/pyunits.mol/pyunits.K**3),
                            'D': (4.911E-8, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "cp_mol_liq_comp_coeff": {
                            '1': (1.40E2, pyunits.J/pyunits.kmol/pyunits.K),  # [2]
                            '2': (-1.52E-1, pyunits.J/pyunits.kmol/pyunits.K**2),
                            '3': (6.95E-4, pyunits.J/pyunits.kmol/pyunits.K**3),
                            '4': (0, pyunits.J/pyunits.kmol/pyunits.K**4),
                            '5': (0, pyunits.J/pyunits.kmol/pyunits.K**5)},
                        "enth_mol_form_liq_comp_ref": (
                            12.0e3, pyunits.J/pyunits.mol),  # [3]
                        "enth_mol_form_vap_comp_ref": (
                            50.1e3, pyunits.J/pyunits.mol),  # [3]
                        "pressure_sat_comp_coeff": {'A': (-7.28607, None),  # [1]
                                                    'B': (1.38091, None),
                                                    'C': (-2.83433, None),
                                                    'D': (-2.79168, None)}}}},

    # Specifying phases
    "phases":  {'Liq': {"type": LiquidPhase,
                        "equation_of_state": Cubic,
                        "equation_of_state_options": {
                            "type": CubicType.PR}},
                'Vap': {"type": VaporPhase,
                        "equation_of_state": Cubic,
                        "equation_of_state_options": {
                            "type": CubicType.PR}}},

    # Set base units of measurement
    "base_units": {"time": pyunits.s,
                   "length": pyunits.m,
                   "mass": pyunits.kg,
                   "amount": pyunits.mol,
                   "temperature": pyunits.K},

    # Specifying state definition
    "state_definition": FTPx,
    "state_bounds": {"flow_mol": (0, 100, 1000, pyunits.mol/pyunits.s),
                     "temperature": (273.15, 300, 450, pyunits.K),
                     "pressure": (5e4, 1e5, 1e6, pyunits.Pa)},
    "pressure_ref": (1e5, pyunits.Pa),
    "temperature_ref": (300, pyunits.K),

    # Defining phase equilibria
    "phases_in_equilibrium": [("Vap", "Liq")],
    "phase_equilibrium_state": {("Vap", "Liq"): SmoothVLE},
    "bubble_dew_method": LogBubbleDew,
    "parameter_data": {"PR_kappa": {("benzene", "benzene"): 0.000,
                                    ("benzene", "toluene"): 0.000,
                                    ("toluene", "benzene"): 0.000,
                                    ("toluene", "toluene"): 0.000}}}

### 2.4 Building the model

In the next cell, we will first create a model and attach a the property package. 

In [8]:
def PR_model(data):
    
    m = ConcreteModel()
    
    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(368)
    m.fs.state_block.pressure.fix(101325)
    m.fs.state_block.mole_frac_comp["benzene"].fix(0.5)
    m.fs.state_block.mole_frac_comp["toluene"].fix(0.5)
    
    m.fs.properties.PR_kappa['benzene', 'toluene'].fix(0.0)
    m.fs.properties.PR_kappa['toluene', 'benzene'].fix(0.0)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
       
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa['benzene', 'toluene'].setlb(-3)
    m.fs.properties.PR_kappa['benzene', 'toluene'].setub(3)

    m.fs.properties.PR_kappa['toluene', 'benzene'].setlb(-3)
    m.fs.properties.PR_kappa['toluene', 'benzene'].setub(3)
  
    # Return initialized flash model
    return m

In [9]:
import pytest

test_data = {"temperature": 368}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-02-18 14:44:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 0


In [10]:
variable_name = ["fs.properties.PR_kappa['benzene', 'toluene']",
                "fs.properties.PR_kappa['toluene', 'benzene']"]

In [11]:
def SSE(m, data):
    expr = ((float(data["vap_benzene"]) -
             m.fs.state_block.mole_frac_phase_comp["Vap", "benzene"])**2 +
            (float(data["liq_benzene"]) -
             m.fs.state_block.mole_frac_phase_comp["Liq", "benzene"])**2)
    return expr*1E4

In [12]:
# solver_options = {'tol': 1e-7}
pest = parmest.Estimator(PR_model, data, variable_name, SSE)

obj_value, parameters = pest.theta_est()

2022-02-18 14:44:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:44:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:44:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:45:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:45:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:45:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:45:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:45:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-18 14:45:23 [INFO] idaes

In [13]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 4.558301

The values for the parameters are as follows:
fs.properties.PR_kappa[benzene,toluene] = 0.005800112291889032
fs.properties.PR_kappa[toluene,benzene] = 0.011792015963987386
